### Importing libraries

In [1]:
import openravepy
import trajoptpy
import json
import numpy as np
import trajoptpy.kin_utils as ku
from trajoptpy.check_traj import traj_is_safe
import time
import random 
import matplotlib.pyplot as plt
import pickle
from trajopt_util import *
from planning_util import *
from regression import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

FILENAME = 'data/dual_arm_fixed_init/'

/home/teguh/.local/lib/python2.7/site-packages/h5py/__init__.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/home/teguh/.local/lib/python2.7/site-packages/h5py/__init__.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import h5a, h5d, h5ds, h5f, h5fd, h5g, h5r, h5s, h5t, h5p, h5z
/home/teguh/.local/lib/python2.7/site-packages/h5py/_hl/group.py:22: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .. import h5g, h5i, h5o, h5r, h5t, h5l, h5p
 /home/teguh/.local/lib/python2.7/site-packages/sklearn/utils/__init__.py:9: RuntimeWarning:numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
 /home/teguh/.local/lib/python2.7/site-packages/sklearn/utils/extmath.py:24: RuntimeWarning:numpy.dtype size changed, may indicate bina

### Setting the trajopt

In [3]:
np.set_printoptions(precision=4, suppress=True)

env = openravepy.Environment()
env.StopSimulation()
env.Load("robots/pr2-beta-static.zae")
#

In [69]:
env.Load("../env/bookshelves.env.xml")
#env.SetDefaultViewer()

True

### Setting the robot

In [70]:
robot = env.GetRobots()[0]
left_arm = robot.GetManipulator("leftarm")
right_arm = robot.GetManipulator("rightarm")

robot.SetActiveDOFs(np.r_[right_arm.GetArmIndices(), left_arm.GetArmIndices()])
coeffs = [1,1,1,1,1,1,1,  1,1,1,1,1,1,1,] #coefficients for joint velocity cost

dof = len(robot.GetActiveDOFValues())
left_dof_away = np.array([ 0.5646,  1.1371, -0.65  , -2.1172,  2.0552, 0.99, -2.168 ])
right_dof_away = np.array([ -0.5646,  1.1371, 0.65  , -2.1172,  2.0552, 0.99, -2.168 ])
robot.SetDOFValues(left_dof_away, left_arm.GetArmIndices())
robot.SetDOFValues(right_dof_away, right_arm.GetArmIndices())
init_joint0 = robot.GetActiveDOFValues()

In [71]:
from openravepy import *
planner = RaveCreatePlanner(env, 'OMPL_RRT')
simplifier = RaveCreatePlanner(env, 'OMPL_Simplifier')

# Setup the planning instance.
params = Planner.PlannerParameters()
params.SetRobotActiveJoints(robot)

In [82]:
robot.SetActiveDOFs(np.r_[right_arm.GetArmIndices()])

In [83]:
cur_joints = robot.GetActiveDOFValues()
print cur_joints

[-1.3955  0.908   0.0103 -0.3979 -1.5615 -0.7265  0.8617]


In [84]:
limits = dict()
limits[0] = [[0.47, 0.8],[-0.5, 0.5],[0.3,1.4]]

In [89]:
goal_joints

array([-1.0006,  1.2477, -1.4897, -1.2559,  1.3275, -0.4728, -0.9384])

In [93]:
goal_joints,_,_,_,_ = get_random_pose_both(env,limits,right_arm, left_arm)

#goal_joints = cur_joints + np.random.rand(len(cur_joints))*0.4
robot.SetActiveDOFValues(goal_joints)
print robot.CheckSelfCollision()

False


In [94]:
robot.SetActiveDOFValues(cur_joints)


print planner.SendCommand('GetParameters')

planner.InitPlan(robot, params)

goal_bias (0.:.05:1.)
range (0.:1.:10000.)



True

In [97]:
params.SetInitialConfig(cur_joints)
params.SetGoalConfig(goal_joints)

# Set the timeout and planner-specific parameters. You can view a list of
# supported parameters by calling: planner.SendCommand('GetParameters')
print 'Parameters:'
print planner.SendCommand('GetParameters')

params.SetExtraParameters('<range>0.2</range>')
params.SetExtraParameters('<goal_bias>0.2</goal_bias>')

print 'Calling the OMPL_RRTConnect planner.'
traj = RaveCreateTrajectory(env, '')
planner.InitPlan(robot, params)
tic = time.time()
result = planner.PlanPath(traj)
toc = time.time()
print toc-tic
assert result == PlannerStatus.HasSolution

Parameters:
goal_bias (0.:.05:1.)
range (0.:1.:10000.)

Calling the OMPL_RRTConnect planner.
10.0914878845


AssertionError: 

In [67]:
n = traj.GetNumWaypoints()
traj_cur = traj.GetWaypoints(0,n)

traj_cur = traj_cur.reshape(-1,14)

print traj_cur.shape

(202, 14)


In [68]:
import time
for t in traj_cur:
    robot.SetActiveDOFValues(t)
    time.sleep(0.4)

KeyboardInterrupt: 

In [45]:
traj_cur.shape

(2, 14)

### Set the IK Solver 

In [13]:
hmat_target = np.array([[ 0.9689,  0.    , -0.2474,  0.4918],
       [-0.    ,  1.    ,  0.    ,  0.912 ],
       [ 0.2474, -0.    ,  0.9689,  0.8947],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])
quat_target = openravepy.quatFromRotationMatrix(hmat_target).tolist()
xyz_target = hmat_target[0:3,3].tolist()

# BEGIN IK for all manips
manips = [left_arm,right_arm]
for manip in manips:
    robot.SetActiveManipulator(manip)
    ikmodel = openravepy.databases.inversekinematics.InverseKinematicsModel(
        robot, iktype=openravepy.IkParameterization.Type.Transform6D)
    if not ikmodel.load():
        ikmodel.autogenerate()   
    
target_joint = ku.ik_for_link(hmat_target, left_arm, "l_gripper_tool_frame",
    filter_options = openravepy.IkFilterOptions.CheckEnvCollisions)
# END ik

### Construct a random problem 

In [13]:
from planning_util import *
limits = dict()
limits[0] = [[0.47, 0.8],[-0.5, 0.5],[0.3,1.4]]
for i in range(1):
    _,_,_,_,target_joint = get_random_pose_both(env,limits, right_arm, left_arm)
    robot.SetActiveDOFValues(target_joint)

NameError: global name 'ku' is not defined

### Build / Retrieve Dataset

In [ ]:
def build_database(x_init, y_init, num_traj, limits, n_steps = 30, predictor=None):
    num_plan = 0
    comp_times = []
    costs = []

    tic = time.time()
    while len(x_init) < num_traj:
        print('{}th Planning, {} trajectories in database'.format(num_plan,len(x_init)))
        #Get a random pose as target
        target_joint_right,xyz_target_right,target_joint_left, \
                            xyz_target_left,target_joint = get_random_pose_both(env,limits,right_arm,left_arm)
        robot.SetActiveDOFValues(target_joint)
        
        #Build memory using a set of predictors
        for func in predictor:
            #Predict the warmstart
            x_cur = np.concatenate([target_joint])[None,:]
            if isinstance(func, Straight_Regressor):
                traj,_ = func.predict(init_joint0, target_joint)     
            else:
                traj,_ = func.predict(x_cur)
                
            if func.is_transform == 'PCA':
                traj = func.pca.inverse_transform(traj)
            
            traj = traj.reshape(-1,dof)
            traj[0,:] = init_joint0
            
            #Plan
            robot.SetActiveDOFValues(init_joint0)
            request_traj = define_request(time_step=n_steps, coeffs = coeffs,init_type='given_traj',constraint_type='joint')
            request_traj = add_constraint(request_traj, 'joint', '', target_joint,-1)
            request_traj = set_init(request_traj, 'given_traj', traj)
            duration, result = run_opt(request_traj, env)    

            #Check traj result
            traj = result.GetTraj()
            if check_traj(env,result, target_joint):
                print 'Planning is successfull!'
                x_init.append(np.concatenate(x_cur))
                y_init.append(traj.flatten())
                comp_times.append(duration)
                costs.append(result.GetCosts()[0][1])
                break
            else:
                print('Fail to find good solution!')
                continue        

        num_plan += 1
        
    
    #Store the result
    toc = time.time()
    total_time = toc-tic
    success_rate = num_traj*1.0/num_plan
    x_init = np.vstack(x_init)
    y_init = np.vstack(y_init)
    
    data = dict()
    data['x'] = x_init
    data['y'] = y_init
    data['total_time'] = total_time
    data['success_rate'] = success_rate
    data['comp_times'] = comp_times
    data['costs'] = costs
    return data

In [ ]:
to_build_database = False
to_augment_database = False
num_traj = 500
n_steps = 30
limits = dict()
limits[0] = [[0.47, 0.8],[-0.5, 0.5],[0.3,1.4]]

#build using straight-line warm-start
straight = Straight_Regressor(dof = dof, n_steps = n_steps)

if to_build_database:
    if to_augment_database:
        #load data
        data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
        x_init = list(data['x'])
        y_init = list(data['y'])
        num_traj += len(x_init)     
    else:
        x_init,y_init = [],[]
        
    data = build_database(x_init, y_init, num_traj,limits, n_steps = n_steps, predictor=[straight])
    x_init = data['x']
    y_init = data['y']
    pickle.dump(data,open(FILENAME + 'data.pkl', 'wb') )
    print('Success_rate : {}, average costs:{}'.format(data['success_rate'], np.mean(data['costs'])))
else:
    #load data
    data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
    x_init = data['x']
    y_init = data['y']
    num_traj = len(x_init)
    print('Obtain {} trajectories'.format(num_traj))

### Reduce the data sample

In [ ]:
x_init = x_init[0:200]
y_init = y_init[0:200]

### For Report 

### Apply Dimensionality Reduction to the data

In [ ]:
from sklearn.decomposition import PCA
y_pca = PCA(n_components = 50)
y_pca.fit(y_init)
y_init_reduced = y_pca.transform(y_init)

### Check Database 

### Reduce the data sample

### Build Function Approximator

In [ ]:
dim_input = len(x_init[0])
is_load_regressor = False

straight = Straight_Regressor(dof = dof, n_steps=n_steps)
nn = NN_Regressor()
gpy = GPy_Regressor(dim_input = dim_input)
gpy_pca = GPy_Regressor(dim_input = dim_input, is_transform='PCA')
gpy_pca.pca = y_pca
bgmr = DP_GLM_Regressor()
bgmr_pca = DP_GLM_Regressor(is_transform='PCA')
bgmr_pca.pca = y_pca
if is_load_regressor:
    nn.load_from_file(FILENAME + 'nn')
    gpy.load_from_file(FILENAME + 'gpy')
    gpy_pca.load_from_file(FILENAME + 'gpy_pca')
    bgmr.load_from_file(FILENAME + 'bgmr')
    bgmr_pca.load_from_file(FILENAME + 'bgmr_pca')
else:
    print 'Planning for NN'
    nn.fit(x_init, y_init)
    nn.save_to_file(FILENAME + 'nn')

    print 'Planning for GPY'
    gpy.fit(x_init, y_init, num_restarts=10)
    gpy.save_to_file(FILENAME + 'gpy')
    
    print 'Planning for GPY PCA'
    gpy_pca.fit(x_init, y_init_reduced)
    gpy_pca.save_to_file(FILENAME + 'gpy_pca')

    print 'Planning for bgmr'
    bgmr.fit(x_init,y_init)
    bgmr.save_to_file(FILENAME + 'bgmr')
    
    print 'Planning for bgmr PCA'
    bgmr_pca.fit(x_init,y_init_reduced)
    bgmr_pca.save_to_file(FILENAME + 'bgmr_pca')

### Creating the test cases

In [ ]:
create_test = False

if create_test:
    x_test = []
    target_joints = []
    num_test = 500
    for i in range(num_test):
        target_joint_right,xyz_target_right,target_joint_left,xyz_target_left,target_joint = get_random_pose_both(env,limits,right_arm,left_arm)
        robot.SetActiveDOFValues(target_joint)
        cur_x = target_joint
        x_test.append(cur_x)

    x_test = np.vstack(x_test)
    data_test = dict()
    data_test['x_test'] = x_test
    test_file = open(FILENAME + 'data_test.pkl', 'wb')
    pickle.dump(data_test,test_file)
    test_file.close()
else:
    #load data
    test_file = open(FILENAME + 'data_test.pkl', 'rb')
    data_test = pickle.load(test_file)
    x_test = data_test['x_test']
    num_test = len(x_test)
    test_file.close()

### Testing the warmstart

In [ ]:
method_names = ['STD    ', 'NN     ', 'GPR    ', 'GPR_PCA', 'BGMR', 'BGMR_PCA']
methods = [straight, nn, gpy, gpy_pca,  bgmr, bgmr_pca]
results = dict()
result,success, duration = dict(),dict(),dict()
for method in method_names:
    results[method] = dict()
    results[method]['costs'] = []
    results[method]['successes'] = []
    results[method]['comp_times'] = []
    results[method]['func_evals'] = []
    results[method]['qp_solves'] = []
    
num_test = 10
for i in range(num_test):
    print('{}th Planning'.format(i))
    
    #setting up the problem case
    index = np.random.randint(0,len(x_test))
    index = i
    x_cur = x_test[index:index+1,:]
    target_joint = x_cur.copy().flatten()
    robot.SetActiveDOFValues(init_joint0)
    
    #plan with various warmstarts
    request_traj = define_request(time_step=n_steps, coeffs = coeffs,init_type='given_traj',constraint_type='joint')
    request_traj = add_constraint(request_traj, 'joint', '', target_joint,-1)
    
    for i,method in enumerate(methods):
        method_name = method_names[i]
        if isinstance(method, Straight_Regressor):
            traj,cov = method.predict(init_joint0, target_joint)
        else:
            traj,cov = method.predict(x_cur)
            
        if method.is_transform == 'PCA':
            traj = method.pca.inverse_transform(traj)
            
        traj = traj.reshape(-1,dof)
        traj[0,:] = init_joint0
        robot.SetActiveDOFValues(init_joint0)
        request_traj = set_init(request_traj, 'given_traj', traj)
        duration[method_name], result[method_name] = run_opt(request_traj, env)    
        success[method_name] = check_traj(env,result[method_name], target_joint)

    #Record the result
    for method_name in method_names:
        results[method_name]['costs'].append(result[method_name].GetCosts()[0][1])
        results[method_name]['func_evals'].append(result[method_name].GetNumFuncEval()[0])
        results[method_name]['qp_solves'].append(result[method_name].GetNumQPSolve()[0])
        results[method_name]['successes'].append(success[method_name])      
        results[method_name]['comp_times'].append(duration[method_name])


### Printing Results

In [ ]:
print(' Method \t| Success Rate \t| Conv. Time \t| Traj. Cost \t| Func. Evals \t| QP Solves')
for method in method_names:
    successes = np.array(results[method]['successes'])
    success = np.count_nonzero(successes)
    
    comp_times = np.array(results[method]['comp_times'])[successes]
    costs = np.array(results[method]['costs'])[successes]
    func_evals = np.array(results[method]['func_evals'])[successes]
    qp_solves = np.array(results[method]['qp_solves'])[successes]
    
    print('{0} \t& {1:.3f} \t& {2:.2f} \t& {3:.3f} \t \\\\'.format(method, success*100.0/len(successes), np.mean(comp_times), np.sum(costs)/success))#, 1.0*np.sum(func_evals)/success, 1.0*np.sum(qp_solves)/success ))
    

### Printing Results Tied to Standard Success

In [ ]:
print(' Method \t| Success Rate \t| Conv. Time \t| Traj. Cost \t| Func. Evals \t| QP Solves')
standard_successes = np.array(results['STD    ']['successes'])

for method in method_names:
    successes = np.array(results[method]['successes'])[standard_successes]
    success = np.count_nonzero(successes)
    comp_times = np.array(results[method]['comp_times'])[standard_successes][successes]
    costs = np.array(results[method]['costs'])[standard_successes][successes]
    func_evals = np.array(results[method]['func_evals'])[standard_successes][successes]
    qp_solves = np.array(results[method]['qp_solves'])[standard_successes][successes]
    
    print('{0}: \t {1:.3f} \t {2:.2f} \t {3:.3f} \t {4:.3f} \t {5:.3f}'.format(method, success*1.0/len(successes), np.sum(comp_times)/success, np.sum(costs)/success, 1.0*np.sum(func_evals)/success, 1.0*np.sum(qp_solves)/success ))
    